In [110]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras # keras
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import warnings
import re

import bs4

warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

In [111]:
#Twitter API
!pip install tweepy==3.3.0

In [7]:
!pip install textblob
!python -m textblob.download_corpora

    100% |████████████████████████████████| 634kB 31.2MB/s ta 0:00:01
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
[nltk_data] Downloading package brown to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/movie_review

In [65]:
!pip install BeautifulSoup4

In [112]:
import tweepy
from tweepy import OAuthHandler
 
consumer_key = 'QlfNT98yekK45kbq2SitmzgvT'
consumer_secret = 'UQHw9sxRUcZ9WlcneEqAEaam5jFCyIfltErfCmJb6owU6k2Gxx'
access_token = '266344008-AdoxTIaIcU4CEwJEUPy4IJAlO3gbjy1zl0CSJek0'
access_secret = 'yn9MvzKV5irL6UKJmFHDu9pZGEeyQAbfIy15ntg6aTQo0'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

def process_or_store(tweet):
    print(json.dumps(tweet))

In [113]:
for status in tweepy.Cursor(api.home_timeline).items(10):
    # Process a single status
    print(status.text)

GOP Sen. Jeff Flake on Khashoggi’s killing: “There are things that we’re going to have to confront here soon, and I… https://t.co/dWE9MNTCVP
Acclaimed screenwriter and novelist William Goldman dies at 87: https://t.co/dR5h0kQryr https://t.co/3Qb8G523hQ
How space experts like Elon Musk convinced Ron Howard that humans should go to Mars https://t.co/cABGebFjPC by… https://t.co/JJG3mbozRN
it turns out our planet hasn't entirely cornered the market on the beauty of a new day dawning.… https://t.co/8gIiCsGojE
The fires in California have consumed entire communities, but also activated the bonds of those communities... https://t.co/XVqUgSMkjO
Same work load, more fatigue? Here's why coffee isn't going to fix it (and what will). https://t.co/3N60jaJKNM
We can't just change our coffee cups, we have to change our lives https://t.co/FgZ0KIpJTC https://t.co/eMSSKIsnvM
On the World Day of the Poor, #PopeFrancis said Christians must go down to meet the needs of others after having go… https://t.co/

In [41]:
!pip install dataset

    100% |████████████████████████████████| 1.0MB 24.6MB/s ta 0:00:01
    100% |████████████████████████████████| 573kB 32.4MB/s ta 0:00:01
  Running setup.py bdist_wheel for alembic ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/57/58/91/31e495c790b05e82563181064a9670168d7eeb817e5595e3c3
  Running setup.py bdist_wheel for Mako ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/15/35/25/dbcb848832ccb1a4b4ad23f529badfd3bce9bf88017f7ca510
  Running setup.py bdist_wheel for python-editor ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/36/e0/98/ba386b125a00ea9dd52e2c16aa2ec0adbbd639b84bfe2e001d
Successfully built alembic Mako python-editor


In [49]:
!pip install datafreeze

In [118]:
##LIVE STREAMING##

from __future__ import absolute_import, print_function

# Import modules
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import dataset
from sqlalchemy.exc import ProgrammingError

class MyListener(StreamListener):
 
    '''
    def on_data(self, data):
        try:
            with open('python.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
    '''
    def on_status(self, status):
        print(status.text)
        if status.retweeted:
            return

        id_str = status.id_str
        created = status.created_at
        text = status.text
        fav = status.favorite_count
        name = status.user.screen_name
        description = status.user.description
        loc = status.user.location
        user_created = status.user.created_at
        followers = status.user.followers_count

        table = db['myTable']

        try:
            table.insert(dict(
                id_str=id_str,
                created=created,
                text=text,
                fav_count=fav,
                user_name=name,
                user_description=description,
                user_location=loc,
                user_created=user_created,
                user_followers=followers,
            ))
        except ProgrammingError as err:
            print(err)

    def on_error(self, status):
        print(status)
        return True

#results = Stream(auth, MyListener())

if __name__ == '__main__':
    db = dataset.connect("sqlite:///tweets.db")
    l = MyListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)

    stream = Stream(auth, l)
    filtered = stream.filter(track=['MBTA'])
#, locations=[-72,42,-71,43]

# The search term you want to find
#query = "MBTA"

# Calling the user_timeline function with our parameters
#results = api.search(q=query)

#locations=[-72,42,-71,43]

# foreach through all tweets pulled
for tweet in filtered:
   # printing the text stored inside the tweet object
   print(tweet.user.screen_name,"Tweeted:",tweet.text)

What are other examples like this? How is the MBTA potentially perpetuating inequities? Or otherwise sucking in way… https://t.co/dCFJRShLsX
RT @bluemassgroup: Does it matter that @MassGovernor refuses to take the T? Yes, because then he’d know how it fails in day-to-day, operati…
Bus 86 Reservoir is running 24 minutes late.  The bus was scheduled to be at Western Ave @ Everett St at 9:12 but i… https://t.co/1Km9an3ybF
Route 8 experiencing delays of up to 20 minutes due to traffic #mbta
Route 8 experiencing delays of up to 20 minutes due to traffic #mbta
Route 77 experiencing delays of up to 15 minutes due to traffic #mbta
RT @crystalgeyser93: Transportation idea: horse pedaling a tandem bike, rider on top of horse steers handlebars with reins @MBTA
RT @RegoTJ: BREAKING: Uber begins test partnership with the @MBTA for more efficient #GreenLine service https://t.co/SSMsWkQ13I
Bus 23 Ashmont is running 42 minutes late.  The bus was scheduled to be at Talbot Ave @ Lithgow St at 9:03 but i

KeyboardInterrupt: 

In [119]:
import dataset
from datafreeze import freeze

db = dataset.connect("sqlite:///tweets.db")
result = db['myTable'].all()
freeze(result, format='csv', filename='tweets10.csv')


In [93]:
import csv

csvFile = open('FriSatTweets.csv', 'a')
csvWriter = csv.writer(csvFile)
table = db['FriSat']

search_terms = ('@MBTA')

for status in tweepy.Cursor(api.search,
                       q=search_terms,
                       since='2018-11-16', until='2018-11-17',
                       count=100,
                       result_type='recent',
                       include_entities=True,
                       monitor_rate_limit=True, 
                       wait_on_rate_limit=True,
                       #geocode='42.36,-71.06,50mi',
                       lang="en").items():
'''
    try:
            table.insert(dict(
                id_str=id_str,
                created=created,
                text=text,
                fav_count=fav,
                user_name=name,
                user_description=description,
                user_location=loc,
                user_created=user_created,
                user_followers=followers,
            ))
    except ProgrammingError as err:
        print(err)
'''
csvWriter.writerow([status.created_at, status.user.screen_name.encode('utf8'), status.text.encode('utf-8')])
csvFile.close()

IndentationError: expected an indented block (<ipython-input-93-977f998bdaa6>, line 34)

In [89]:
db = dataset.connect("sqlite:///tweets.db")
result = db['FriSat'].all()
freeze(result, format='csv', filename='FriSat.csv')

In [96]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 

class TwitterClient(object): 
	''' 
	Generic Twitter Class for sentiment analysis. 
	'''
	def __init__(self): 
		''' 
		Class constructor or initialization method. 
		'''
		# keys and tokens from the Twitter Dev Console 
		consumer_key = 'XXXXXXXXXXXXXXXXXXXXXXXX'
		consumer_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'
		access_token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'
		access_token_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXX'

		# attempt authentication 
		try: 
			# create OAuthHandler object 
			self.auth = OAuthHandler(consumer_key, consumer_secret) 
			# set access token and secret 
			self.auth.set_access_token(access_token, access_token_secret) 
			# create tweepy API object to fetch tweets 
			self.api = tweepy.API(self.auth) 
		except: 
			print("Error: Authentication Failed") 

	def clean_tweet(self, tweet): 
		''' 
		Utility function to clean tweet text by removing links, special characters 
		using simple regex statements. 
		'''
		return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

	def get_tweet_sentiment(self, tweet): 
		''' 
		Utility function to classify sentiment of passed tweet 
		using textblob's sentiment method 
		'''
		# create TextBlob object of passed tweet text 
		analysis = TextBlob(self.clean_tweet(tweet)) 
		# set sentiment 
		if analysis.sentiment.polarity > 0: 
			return 'positive'
		elif analysis.sentiment.polarity == 0: 
			return 'neutral'
		else: 
			return 'negative'

	def get_tweets(self, query, count = 10): 
		''' 
		Main function to fetch tweets and parse them. 
		'''
		# empty list to store parsed tweets 
		tweets = [] 

		try: 
			# call twitter api to fetch tweets 
			fetched_tweets = self.api.search(q = query, count = count) 

			# parsing tweets one by one 
			for tweet in fetched_tweets: 
				# empty dictionary to store required params of a tweet 
				parsed_tweet = {} 

				# saving text of tweet 
				parsed_tweet['text'] = tweet.text 
				# saving sentiment of tweet 
				parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 

				# appending parsed tweet to tweets list 
				if tweet.retweet_count > 0: 
					# if tweet has retweets, ensure that it is appended only once 
					if parsed_tweet not in tweets: 
						tweets.append(parsed_tweet) 
				else: 
					tweets.append(parsed_tweet) 

			# return parsed tweets 
			return tweets 

		except tweepy.TweepError as e: 
			# print error (if any) 
			print("Error : " + str(e)) 

def main(): 
	# creating object of TwitterClient Class 
	api = TwitterClient() 
	# calling function to get tweets 
	tweets = api.get_tweets(query = 'Donald Trump', count = 200) 

	# picking positive tweets from tweets 
	ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
	# percentage of positive tweets 
	print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
	# picking negative tweets from tweets 
	ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
	# percentage of negative tweets 
	print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
	# percentage of neutral tweets 
	print("Neutral tweets percentage: {} % \ ".format(100*len(tweets - ntweets - ptweets)/len(tweets))) 

	# printing first 5 positive tweets 
	print("\n\nPositive tweets:") 
	for tweet in ptweets[:10]: 
		print(tweet['text']) 

	# printing first 5 negative tweets 
	print("\n\nNegative tweets:") 
	for tweet in ntweets[:10]: 
		print(tweet['text']) 

if __name__ == "__main__": 
	# calling main function 
	main() 


Error : Twitter error response: status code = 401


TypeError: 'NoneType' object is not iterable

In [102]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')


In [101]:
import json
import requests


payload = {'t':'microblogs', 'from_ts':'86400', 'q':'MBTA Kenmore'}

r = requests.get('http://socialmention.com/search', params=payload)
r.headers['content-type']



'text/html; charset=UTF-8'

In [103]:
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <!-- Start of Woopra Code -->
  <script>
   (function(){
       var t,i,e,n=window,o=document,a=arguments,s="script",r=["config","track","identify","visit","push","call","trackForm","trackClick"],c=function(){var t,i=this;for(i._e=[],t=0;r.length>t;t++)(function(t){i[t]=function(){return i._e.push([t].concat(Array.prototype.slice.call(arguments,0))),i}})(r[t])};for(n._w=n._w||{},t=0;a.length>t;t++)n._w[a[t]]=n[a[t]]=n[a[t]]||new c;i=o.createElement(s),i.async=1,i.src="//static.woopra.com/js/w.js",e=o.getElementsByTagName(s)[0],e.parentNode.insertBefore(i,e)
})("woopra");

woopra.config({
   domain: 'socialmention.com'
});
woopra.track();
  </script>
  <!-- End of Woopra Code -->
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   MBTA Kenmore - Page 1 - Social Mention search
  </ti

In [109]:
text = soup.find_all("div", "sort_results")